In [1]:


from GenZ import decode_moddeling, prefill_moddeling, get_configs

import pandas as pd
from plotnine import *
import plotnine as p9
import pandas as pd
from GenZ.analyse_model import *


In [2]:
# This will reload the imported modules (e.g. get_decode_model_characterstics) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2   

In [3]:
D = get_configs('GPT-4').hidden_size
V = 50257
model_df, summary_table = decode_moddeling(model = 'gpt-4', batch_size = 1, Bb = 1 , bits='int8',
                            input_tokens = 1, output_tokens = 0, model_profilling=True,
                            tensor_parallel = 1, pipeline_parallel = 1, debug=True) 
display(model_df)
display(summary_table)
# print(f'Model Weights={summary_table['Model Weights (MB)'][0]*2**20/1e12} T')
# print(f'Attn Size:{(model_df.loc[0,'Input_w (MB)'] + model_df.loc[5,'Input_w (MB)'])*120*2**20/1e9} B')
# print(f'Each MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120*2**20/1e9/16} B')
# print(f'FWD pass MLP Size:{sum(model_df.loc[6:,'Input_w (MB)'])*120/1024/16} B')

/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_decode.py:62: UserWarning: All params would not fit on chip. System Memory Cap:40.0 GB , Weights : 1819.125 GB, KV Cache:0.0025634765625 


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,Repeat,128,0,0.0,0.0,0.0,0.0,0.0
1,GEMM,"[((1, 10752, 1), (32256, 10752), (1, 32256, 1))]",0.999876,346.816512,0.010254,330.75,0.030762,330.791016
2,Logit,"((1, 84, 1, 128), (1, 84, 1, 128), (1, 84, 1, 1))",0.498054,0.010752,0.010254,0.010254,0.00008,0.020588
3,Logit,"((1, 84, 1, 128), (1, 84, 0, 128), (1, 84, 1, 0))",0.0,0.0,0.010254,0.0,0.0,0.010254
4,Attend,"((1, 84, 1, 1), (1, 84, 1, 128), (1, 84, 1, 128))",0.498054,0.010752,0.00008,0.010254,0.010254,0.020588
5,Attend,"((1, 84, 1, 0), (1, 84, 0, 128), (1, 84, 1, 128))",0.0,0.0,0.0,0.0,0.010254,0.010254
6,GEMM,"[((1, 10752, 1), (10752, 10752), (1, 10752, 1))]",0.999814,115.605504,0.010254,110.25,0.010254,110.270508
7,GEMM,"[((1, 10752, 1), (86016, 10752), (1, 86016, 1))]",0.999895,924.844032,0.010254,882.0,0.082031,882.092285
8,GEMM,"[((1, 10752, 0), (602112, 10752), (1, 602112, ...",0.0,0.0,0.0,6174.0,0.0,6174.0
9,GEMM,"[((1, 86016, 1), (10752, 86016), (1, 10752, 1))]",0.999895,924.844032,0.082031,882.0,0.010254,882.092285


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB)
0,295952.842752,1.862823e+06,1862784.0,12348.0,2.625,6174.0


In [4]:
for model in ['gpt-4']:
    model_df, summary_table = prefill_moddeling(model = model, batch_size = 1,
                            input_tokens = 1024, model_profilling=True,
                            tensor_parallel = 8, pipeline_parallel = 16) 
    print(model)
    display(model_df)
    display(summary_table)


gpt-4


/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_prefill.py:22: UserWarning: Batch size is divided into micro batches for pipeline parallel, micro batch size:1, consider increasing batch size


,Op Type,Dimension,Op Intensity,Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB)
0,Repeat,128,0,0.0,0.0,0.0,0.0,0.0
1,GEMM,"[((1, 10752, 1024), (4032, 10752), (1, 4032, 1...",379.482353,44392.513536,21.0,82.6875,7.875,111.5625
2,Logit,"((1, 10, 1024, 128), (1, 10, 1024, 128), (1, 1...",51.2,1342.17728,2.5,2.5,20.0,25.0
3,Attend,"((1, 10, 1024, 1024), (1, 10, 1024, 128), (1, ...",51.2,1342.17728,20.0,2.5,2.5,25.0
4,GEMM,"[((1, 1280, 1024), (10752, 1280), (1, 10752, 1...",270.150754,14092.86144,2.5,26.25,21.0,49.75
5,Sync,"(1, 1024, 10752)",0,0.0,0.0,0.0,0.0,0.0
6,GEMM,"[((1, 10752, 128), (86016, 10752), (1, 86016, ...",63.154185,118380.036096,2.625,1764.0,21.0,1787.625
7,GEMM,"[((1, 86016, 128), (10752, 86016), (1, 10752, ...",63.154185,118380.036096,21.0,1764.0,2.625,1787.625
8,Sync,"(1, 1024, 10752)",0,0.0,0.0,0.0,0.0,0.0
9,EndRepeat,128,0,0.0,0.0,0.0,0.0,0.0


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB)
0,3.813501e+07,484680.0,465528.0,0.0,640.0,1787.625


In [5]:
prefill_moddeling(model = 'gpt-4', batch_size = 1,
                                            input_tokens = 2000,
                                            system_name = 'H100_GPU',
                                            bits='int8',
                                            tensor_parallel = 8, pipeline_parallel = 16, debug=False) 

/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_prefill.py:22: UserWarning: Batch size is divided into micro batches for pipeline parallel, micro batch size:1, consider increasing batch size


{'Latency': 111.044776197886,
 'Throughput': 9.0053763377213,
 'Runtime_breakdown': [81.33332687512731,
  1.3252982810920122,
  28.38615104166667],
 'is_offload': False}

In [6]:
decode_moddeling(model = 'gpt-4', batch_size = 1, Bb = 4 ,
                                            input_tokens = 2000, output_tokens = 256, 
                                            system_name = 'H100_GPU',
                                            bits='int8',
                                            tensor_parallel = 8, pipeline_parallel = 16, debug=False) 

/Users/abambhaniya3/GenZ/GenZ/LLM_inference/llm_decode.py:24: UserWarning: Batch size is divided into micro batches for pipeline parallel, micro batch size:1, consider increasing batch size


{'Latency': 18.870783471400443,
 'Throughput': 52.99197044550624,
 'Runtime_breakdown': [10.110994924906809,
  0.27191224441029666,
  8.487876302083334],
 'is_offload': False}